# Background

Agent-based models by default use so-called **incremental time progression**. This means that the model clock is advanced by a small time step and the state of the model is updated by activating all agents. A major drawback of this is that it can become computationally very expensive for large models with many agents. Moreover, it can be a wasteful approach is it is knoweable in advance what the update will be. 

For example, in Epstein's civil violence model, if an agent is jailed, every time step, the agent is activated only the reduce the `jail_time_remaining` by 1. However, at the moment of going to jail, it is known when the agent will be released again. So, it would be convenient if it were possible to not active the jailed agents until they are released from prison.

Other examples abound. For example, in pandemic simulations, it makes no sense to activate agents when they are at work or say sleeping. In transportation ABMs, likewise, it makes little sense to check each agent, say, every 15 minutes to see if they want to travel. 

An alternative to **incremental time progression** is **next-event time progression**. In **next-event time progression**, model state updates are done through events. Each event is time stamped. Events are kept in a sorted data structure and the clock advances to the time stamp of the next event, executes the event (which might result in new events being added to the sorted data structure, or *scheduled*). 

So, in case of the Epstein model, when an agent is arested, one might schedule its *release from jail* event for the current time step plus the jail time duration. While other non-jailed agents just schedule themselves for the current time + 1. Likewise, in pandemic models or transportation models, agents would schedule when they will travel and won't be activated until that travel event is executed.

It is also possible to combine **incremental time progression** and **next-event time progression**. That is, one would use **next-event time progression** as is normal in ABMs but add the possibility of scheduling events to occur at given time steps. This offers a flexible hybrid solution, easy to add to existing ABMs, while potentially substantially reducing runtime. 

Mesa comes with support for both pure **next-event time progression** and this hybrid approach. It's available via `mesa.experimenta.devs`. `devs` stands for discrete event simulation, which is the name of a broad family of simulation methods. ABMs are in essence a particular type of discrete event simulation, but other formalisms next to ABM exist. 

Below, I have adapted the Epstein model to use the hybrid approach. In short
1. I added a `model.free_citizens` attribute, which is an agentset containing the non-jailed agents.
2. In `model.step` I only activate the free citizens and the cops
3. In the `Citizen`, I added 2 methods: `arrest` and `release`. In `arrest`, I set the state to `ARRESTED`, schedule the release method for the jail time, and remove the citizen from `model.free_citizen`. In `release`, we set the state to `QUIET` and add the agent back to `model.free_citizens`.
4. I modified the `Cop` step method to simply call `Citizen.arrest` with the random jail time.
5. The way of running a model is a bit different. The run is controlled by an ABMSimulator rather than us calling `model.step`. In the provided code, I run the model for 100 timesteps.


In [1]:
import math
from enum import Enum

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.axes_grid1 import make_axes_locatable

from mesa import Model, Agent
from mesa.datacollection import DataCollector
from mesa.discrete_space import OrthogonalVonNeumannGrid, CellAgent


class CitizenState(Enum):
    ACTIVE = 1    
    QUIET = 2    
    ARRESTED = 3


class CivilViolence(Model):
    """Model class for Eppstein's Civil Violence model I.
    
    The initial values are from Eppstein's article.
    """

    def __init__(self, simulator, height=40, width=40, citizen_density=0.7, citizen_vision=7,
        legitimacy=0.82, activation_treshold=0.1, arrest_prob_constant=2.3,
        cop_density=0.04, cop_vision=7, max_jail_term=15, seed=None):
        super().__init__(seed=seed)
        self.simulator = simulator

        assert (citizen_density+cop_density) < 1
        
        # setup Citizen class attributes
        Citizen.vision = citizen_vision
        Citizen.legitimacy = legitimacy
        Citizen.arrest_prob_constant = arrest_prob_constant
        Citizen.activation_threshold = activation_treshold
        
        # setup Cop class attributes
        Cop.vision = cop_vision
        Cop.max_jail_term = max_jail_term

        # setup data collection
        model_reporters = {'active': CitizenState.ACTIVE.name,
                           'quiet': CitizenState.QUIET.name,
                           'arrested': CitizenState.ARRESTED.name}

        # populate agents
        self.ACTIVE = 0
        self.QUIET = 0
        self.ARRESTED = 0
        
        
        self.datacollector = DataCollector(model_reporters=model_reporters)
        self.grid = OrthogonalVonNeumannGrid((width, height), capacity=1,
                                        torus=True, random=self.random)

        # Set up agents
        for cell in self.grid.all_cells:
            klass = self.random.choices([Citizen, Cop, None],
                                   cum_weights=[citizen_density,
                                                citizen_density+cop_density, 1])[0]
            if klass: 
                agent = klass(self)
                agent.cell = cell

        self.free_citizens = self.agents_by_type[Citizen]
        
        self._update_counts()
        self.datacollector.collect(self)

    def _update_counts(self):
        for state, count in self.agents_by_type[Citizen].groupby("state").count().items():
            setattr(self, state.name, count)
        
    def step(self):
        """
        Run one step of the model.
        """
        self.free_citizens.shuffle_do("step")
        self.agents_by_type[Cop].shuffle_do("step")
        self._update_counts()        
        self.datacollector.collect(self)


class BaseAgent(CellAgent):
    '''Base Agent class implementing vision and moving
    
    Attributes
    ----------
    moore : boolean
    
    '''    
    
    def get_agents_in_vision(self):
        """
        identify cops and active citizens within vision
        
        Returns
        -------
        tuple with list of cops, and list of active citizens
        
        """
        cops = []
        active_citizens = []
        
        for agent in self.cell.get_neighborhood(radius=self.__class__.vision).agents:
            if isinstance(agent, Cop):
                cops.append(agent)
            elif agent.state == CitizenState.ACTIVE:
                active_citizens.append(agent)    
        return cops, active_citizens
    
    def move(self):
        """Identify all empty cells within vision and move to a randomly selected one."""
        empty = [cell for cell in self.cell.get_neighborhood(radius=self.__class__.vision) 
                 if cell.is_empty]
        
        if empty:
            self.cell = self.random.choice(empty)
        
    
class Citizen(BaseAgent):
    '''Citizen class
    
    Attributes
    ----------
    legitimacy : boolean
    vision : int
    arrest_prob_constant : float
    activation_treshold : float
    hardship : float
    risk_aversion : float
    state : {CitizenState.QUIET, CitizenState.ACTIVE, CitizenState.ARRESTED }
    jail_time_remaining  :int
    grievance : float
    
    '''        
    legitimacy = 1
    vision = 1
    arrest_prob_constant = 1
    activation_treshold = 1
    
    def __init__(self, model):
        super().__init__(model)
        self.hardship = self.random.random()
        self.risk_aversion = self.random.random()
        self.state = CitizenState.QUIET
        self.grievance = self.hardship*(1-Citizen.legitimacy)


    def arrest(self, jail_time):
        self.state = CitizenState.ARRESTED
        self.model.free_citizens.remove(self)
        self.model.simulator.schedule_event_relative(self.release, jail_time)

    def release(self):
        self.state = CitizenState.QUIET
        self.model.free_citizens.add(self)
    
    def step(self):
        """
        move and then decide whether to activate
        """         
        self.move()
            
        cops, active_citizens = self.get_agents_in_vision()
        n_cops = len(cops)
        n_active_citizens = len(active_citizens) + 1 # self is always considerd active
            
        arrest_p = 1 - math.exp(-1*Citizen.arrest_prob_constant * round(n_cops/n_active_citizens))
        net_risk = self.risk_aversion * arrest_p
        
        if (self.grievance - net_risk) > self.activation_threshold:
            self.state = CitizenState.ACTIVE
        else:
            self.state = CitizenState.QUIET
        

class Cop(BaseAgent):
    '''Cop class
    
    Attributes
    ----------
    vision : int
    max_jail_term : int
    '''
    vision = 1
    max_jail_term = 1
        
    def step(self):
        self.move()
        _, active_citizens = self.get_agents_in_vision()
        
        if active_citizens:
            citizen = self.random.choice(active_citizens)
            citizen.arrest(self.random.randint(0, Cop.max_jail_term))



In [2]:
from mesa.experimental.devs import ABMSimulator

simulator = ABMSimulator()
model = CivilViolence(
    simulator,
    seed=15,
)

simulator.setup(model)
simulator.run_for(100)

# Question 1
Compare the runtime of this hybrid event scheduling with incremental time progression version of the model to the model from last week. You can use the time module for this as shown below, or use a `%%timeit` jupyter cell magic. 

```python
import time

start_time = time.perf_counter()
simulator.run_for(100)
print("Time:", time.perf_counter() - start_time)

# or in jupyter lab
%%timeit
simulator.run_for(100)
```

1. How large is the difference in runtime?
2. Can you think of other ways of using event scheduling in this model to further speed up the model?


# Question 2
Below, I give you the wolf sheep grass example model that comes with mesa. This model can also be substantially sped up by using event scheduling. 

1. Adapt the provided model to use event scheduling for the grass regrowth dynamics. That is, whenever the grass get's eaten, it should schedule an event for it to be fully regrown.
2. Compare the runtime between both versions of the model for 100 timesteps, account for the stochastic nature of the model in this runtime comparison. Plot the runtime of both models using histograms.


In [9]:
import mesa
from mesa.discrete_space import CellAgent, FixedAgent, OrthogonalMooreGrid

class Animal(CellAgent):
    """The base animal class."""

    def __init__(self, model, energy, p_reproduce, energy_from_food, cell):
        """Initializes an animal.

        Args:
            model: a model instance
            energy: starting amount of energy
            p_reproduce: probability of sexless reproduction
            energy_from_food: energy obtained from 1 unit of food
            cell: the cell in which the animal starts
        """
        super().__init__(model)
        self.energy = energy
        self.p_reproduce = p_reproduce
        self.energy_from_food = energy_from_food
        self.cell = cell

    def spawn_offspring(self):
        """Create offspring."""
        self.energy /= 2
        self.__class__(
            self.model,
            self.energy,
            self.p_reproduce,
            self.energy_from_food,
            self.cell,
        )

    def feed(self): ...

    def step(self):
        """One step of the agent."""
        self.cell = self.cell.neighborhood.select_random_cell()
        self.energy -= 1

        self.feed()

        if self.energy < 0:
            self.remove()
        elif self.random.random() < self.p_reproduce:
            self.spawn_offspring()


class Sheep(Animal):
    """A sheep that walks around, reproduces (asexually) and gets eaten."""

    def feed(self):
        """If possible eat the food in the current location."""
        # If there is grass available, eat it
        if self.model.grass:
            grass_patch = next(
                obj for obj in self.cell.agents if isinstance(obj, GrassPatch)
            )
            if grass_patch.fully_grown:
                self.energy += self.energy_from_food
                grass_patch.fully_grown = False


class Wolf(Animal):
    """A wolf that walks around, reproduces (asexually) and eats sheep."""

    def feed(self):
        """If possible eat the food in the current location."""
        sheep = [obj for obj in self.cell.agents if isinstance(obj, Sheep)]
        if len(sheep) > 0:
            sheep_to_eat = self.random.choice(sheep)
            self.energy += self.energy_from_food

            # Kill the sheep
            sheep_to_eat.remove()


class GrassPatch(FixedAgent):
    """
    A patch of grass that grows at a fixed rate and it is eaten by sheep
    """

    def __init__(self, model, fully_grown, countdown):
        """
        Creates a new patch of grass

        Args:
            grown: (boolean) Whether the patch of grass is fully grown or not
            countdown: Time for the patch of grass to be fully grown again
        """
        super().__init__(model)
        self.fully_grown = fully_grown
        self.countdown = countdown

    def step(self):
        if not self.fully_grown:
            if self.countdown <= 0:
                # Set as fully grown
                self.fully_grown = True
                self.countdown = self.model.grass_regrowth_time
            else:
                self.countdown -= 1

class WolfSheep(mesa.Model):
    """
    Wolf-Sheep Predation Model
    """

    height = 20
    width = 20

    initial_sheep = 100
    initial_wolves = 50

    sheep_reproduce = 0.04
    wolf_reproduce = 0.05

    wolf_gain_from_food = 20

    grass = False
    grass_regrowth_time = 30
    sheep_gain_from_food = 4

    description = (
        "A model for simulating wolf and sheep (predator-prey) ecosystem modelling."
    )

    def __init__(
        self,
        width=20,
        height=20,
        initial_sheep=100,
        initial_wolves=50,
        sheep_reproduce=0.04,
        wolf_reproduce=0.05,
        wolf_gain_from_food=20,
        grass=False,
        grass_regrowth_time=30,
        sheep_gain_from_food=4,
        seed=None,
    ):
        """
        Create a new Wolf-Sheep model with the given parameters.

        Args:
            initial_sheep: Number of sheep to start with
            initial_wolves: Number of wolves to start with
            sheep_reproduce: Probability of each sheep reproducing each step
            wolf_reproduce: Probability of each wolf reproducing each step
            wolf_gain_from_food: Energy a wolf gains from eating a sheep
            grass: Whether to have the sheep eat grass for energy
            grass_regrowth_time: How long it takes for a grass patch to regrow
                                 once it is eaten
            sheep_gain_from_food: Energy sheep gain from grass, if enabled.
        """
        super().__init__(seed=seed)
        # Set parameters
        self.width = width
        self.height = height
        self.initial_sheep = initial_sheep
        self.initial_wolves = initial_wolves
        self.grass = grass
        self.grass_regrowth_time = grass_regrowth_time

        self.grid = OrthogonalMooreGrid((self.width, self.height), torus=True, random=self.random)

        collectors = {
            "Wolves": lambda m: len(m.agents_by_type[Wolf]),
            "Sheep": lambda m: len(m.agents_by_type[Sheep]),
            "Grass": lambda m: len(
                m.agents_by_type[GrassPatch].select(lambda a: a.fully_grown)
            )
            if m.grass
            else -1,
        }

        self.datacollector = mesa.DataCollector(collectors)

        # Create sheep:
        for _ in range(self.initial_sheep):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            energy = self.random.randrange(2 * self.sheep_gain_from_food)
            Sheep(
                self, energy, sheep_reproduce, sheep_gain_from_food, self.grid[(x, y)]
            )

        # Create wolves
        for _ in range(self.initial_wolves):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            energy = self.random.randrange(2 * self.wolf_gain_from_food)
            Wolf(self, energy, wolf_reproduce, wolf_gain_from_food, self.grid[(x, y)])

        # Create grass patches
        if self.grass:
            for cell in self.grid.all_cells:
                fully_grown = self.random.choice([True, False])

                if fully_grown:
                    countdown = self.grass_regrowth_time
                else:
                    countdown = self.random.randrange(self.grass_regrowth_time)

                patch = GrassPatch(self, fully_grown, countdown)
                patch.cell = cell

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.random.shuffle(self.agent_types)
        for agent_type in self.agent_types:
            self.agents_by_type[agent_type].shuffle_do("step")

        # collect data
        self.datacollector.collect(self)



In [10]:
import time
model = WolfSheep(seed=42)  # this is only 1 deterministic stochastic realization

start_time = time.perf_counter()
for _  in range(100):
    model.step()
print("Time:", time.perf_counter() - start_time)
    

Time: 0.012790790991857648
